In [3]:
import pandas as pd
import numpy as np
import datetime
import calendar
import math
import os
import xarray as xr
import rioxarray
import glob

ouput_folder_path = r"E:\Script\MSP_Data_office_PC\Output\Phytoplankton"

latitude_boundary = [16, 24]
longitude_boundary = [81, 95]

monthly_da_list = []

for file in glob.glob(r"E:\Script\MSP_Data_office_PC\Data\Phytoplankton\CEDA\*.nc"):
    ds = xr.open_dataset(file, engine = 'netcdf4')
    obs_time = pd.to_datetime(ds['time'].data[0])
    
    south_bnd = np.argmin( np.abs(ds['latitude'].data - latitude_boundary[0]))
    north_bnd = np.argmin( np.abs(ds['latitude'].data - latitude_boundary[1]))

    west_bnd = np.argmin( np.abs(ds['longitude'].data - longitude_boundary[0]))
    east_bnd = np.argmin(  np.abs(ds['longitude'].data - longitude_boundary[1]))

    latitude = ds['latitude'].sel(lat=slice(south_bnd, north_bnd+1)).data
    longitude = ds['longitude'].sel(lon=slice(west_bnd, east_bnd+1)).data

    da_name = ds['C_phyto'].sel(lat=slice(south_bnd, north_bnd+1), lon=slice(west_bnd, east_bnd+1))
    
    da_name['lat'] = latitude
    da_name['lon'] = longitude

    monthly_da_list.append(da_name)
    

combined_monthly_da = xr.concat(monthly_da_list , dim = 'time')

# CALCULATING SEASONAL CONCENTRATION AND CREATING SEASONAL DATA ARRAY
winter_months = [time.to_numpy() for time in combined_monthly_da.time if time.dt.month in [1, 2, 12]]
summer_months = [time.to_numpy() for time in combined_monthly_da.time if time.dt.month in [6, 7, 8]]

season_dict = {'Winter': winter_months, 'Summer': summer_months}

season_da_list = []

for season in season_dict.keys():
    
    season_da = combined_monthly_da.sel(time= season_dict[season], lat = slice(latitude_boundary[0], latitude_boundary[1]), 
                                        lon = slice(longitude_boundary[0], longitude_boundary[1]))
    season_mean_da = season_da.mean(dim= 'time', skipna=True, keep_attrs=True)

    season_array = np.array([season], dtype = str)
    
    season_mean_da = season_mean_da.expand_dims(season=season_array, axis=0)
    season_mean_da = season_mean_da.assign_coords(season=season_array)
    
    season_da_list.append(season_mean_da)

combined_sesason_da = xr.concat(season_da_list , dim = 'season')
season_name_array = combined_sesason_da['season'].to_numpy()

# CREATING GEOTIFF FOR EACH SEASON FROM SEASONAL DATAARRAY
for i, season_name in zip(range(0, len(combined_sesason_da['season'])), season_name_array):
    
    season_phyto_da = combined_sesason_da[i]
    
    season_phyto_da.rio.write_crs("epsg:4326", inplace=True)
    season_phyto_da.rio.set_spatial_dims("lon", "lat", inplace=True)
    season_phyto_da.rio.write_nodata(-9999, encoded=True, inplace=True)
    
    season_phyto_da.rio.to_raster(os.path.join(ouput_folder_path, 'Sea_Surface_Phytoplankton_{}_2020.tif'.format(season_name)))

# Saving the montly combined Phytoplankton concentration data array
combined_monthly_da.to_netcdf(os.path.join(ouput_folder_path, 'Monthly_Sea_Surface_Phytoplankton_Combined_2020.nc'))

# Saving the seasonal phytoplankton concentration data array
combined_sesason_da.to_netcdf(os.path.join(ouput_folder_path, 'Seasonal_Sea_Surface_Phytoplankton_Combined_2020.nc'))